In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test

In [2]:
ran = 99

In [3]:
proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
print('Proteins shape:',proteins.shape)
proteins.head()

Proteins shape: (232741, 5)


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [4]:
import warnings
import numpy as np
import pandas as pd

### Machine Learning ###
import xgboost as xgb
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import GroupKFold
### Optimization ###
import optuna

### Viz ###
from colorama import Fore

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [5]:
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
m_ = Fore.MAGENTA
r_ = Fore.RED
y_ = Fore.YELLOW

In [6]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

In [7]:
def smape(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted)
    return 100/len(actual) * np.sum(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))

In [8]:
proteins_test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv')
print('Proteins test shape:',proteins_test.shape)
proteins_test.head()

Proteins test shape: (453, 6)


,visit_id,visit_month,patient_id,UniProt,NPX,group_key
0,50423_0,0,50423,O00391,33127.90,0
1,50423_0,0,50423,O00533,490742.00,0
2,50423_0,0,50423,O00584,43615.30,0
3,50423_0,0,50423,O14773,16486.60,0
4,50423_0,0,50423,O14791,2882.42,0


In [9]:
peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
print('Peptides shape:', peptides.shape)
peptides.head()

Peptides shape: (981834, 6)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [10]:
peptides_test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv')
print('Peptides test shape:', peptides_test.shape)
peptides_test.head()

Peptides test shape: (2057, 7)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,group_key
0,50423_0,0,50423,O00391,AHFSPSNIILDFPAAGSAAR,22226.3,0
1,50423_0,0,50423,O00391,NEQEQPLGQWHLS,10901.6,0
2,50423_0,0,50423,O00533,GNPEPTFSWTK,51499.4,0
3,50423_0,0,50423,O00533,IEIPSSVQQVPTIIK,125492.0,0
4,50423_0,0,50423,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,23174.2,0


In [11]:
clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
print('Clinical shape:', clinical.shape)
clinical.head()

Clinical shape: (2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [12]:
test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv')
print('Test shape:', test.shape)
test

Test shape: (16, 6)


,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0
5,50423_0,0,50423,updrs_2,50423_0_updrs_2,0
6,50423_0,0,50423,updrs_3,50423_0_updrs_3,0
7,50423_0,0,50423,updrs_4,50423_0_updrs_4,0
8,3342_6,6,3342,updrs_1,3342_6_updrs_1,6
9,3342_6,6,3342,updrs_2,3342_6_updrs_2,6


In [13]:
df_0 = clinical[['visit_id','updrs_1']]
print('Train shape:', df_0.shape)
df_0.head()

Train shape: (2615, 2)


,visit_id,updrs_1
0,55_0,10.0
1,55_3,10.0
2,55_6,8.0
3,55_9,8.0
4,55_12,10.0


In [14]:
proteins_npx_ft = proteins.groupby('visit_id').agg(NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std'))\
                .reset_index()
proteins_npx_ft.head()

,visit_id,NPX_min,NPX_max,NPX_mean,NPX_std
0,10053_0,2497.84,269126000.0,2.856580e+06,2.131630e+07
1,10053_12,5800.87,270030000.0,2.728871e+06,2.092162e+07
2,10053_18,1334.11,278835000.0,2.509967e+06,1.969453e+07
3,10138_12,2520.24,365582000.0,3.002583e+06,2.516170e+07
4,10138_24,1436.94,396894000.0,3.068891e+06,2.716806e+07


In [15]:
df_proteins = pd.merge(proteins, df_0, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
proteins_Uniprot_updrs

,UniProt,updrs_1_sum
0,O00391,6.152174
1,O00533,6.568885
2,O00584,6.518483
3,O14498,6.538928
4,O14773,6.560757
...,...,...
222,Q9UHG2,6.566479
223,Q9UKV8,6.545977
224,Q9UNU6,6.517053
225,Q9Y646,6.411706


In [16]:
df_proteins = pd.merge(proteins, df_0, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
proteins_Uniprot_updrs.head()

,UniProt,updrs_1_sum
0,O00391,6.152174
1,O00533,6.568885
2,O00584,6.518483
3,O14498,6.538928
4,O14773,6.560757


In [17]:
df_proteins = pd.merge(proteins, proteins_Uniprot_updrs, on = 'UniProt', how = 'left')
proteins_UniProt_ft = df_proteins.groupby('visit_id').agg(proteins_updrs_1_min=('updrs_1_sum','min'), proteins_updrs_1_max=('updrs_1_sum','max'),\
                                                          proteins_updrs_1_mean=('updrs_1_sum','mean'), proteins_updrs_1_std=('updrs_1_sum','std'))\
                .reset_index()
proteins_UniProt_ft.head()

,visit_id,proteins_updrs_1_min,proteins_updrs_1_max,proteins_updrs_1_mean,proteins_updrs_1_std
0,10053_0,6.152174,6.879464,6.548987,0.076462
1,10053_12,5.981534,6.879464,6.544246,0.088526
2,10053_18,5.689507,6.879464,6.527509,0.111365
3,10138_12,5.689507,6.761074,6.515967,0.120601
4,10138_24,5.689507,6.879464,6.518794,0.121644


In [18]:
s_clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')

In [19]:
peptides_PeptideAbundance_ft = peptides.groupby('visit_id').agg(Abe_min=('PeptideAbundance','min'), Abe_max=('PeptideAbundance','max'),\
                                                                Abe_mean=('PeptideAbundance','mean'), Abe_std=('PeptideAbundance','std'))\
                .reset_index()
peptides_PeptideAbundance_ft.head()

,visit_id,Abe_min,Abe_max,Abe_mean,Abe_std
0,10053_0,82.9679,66333900.0,726248.393431,3.535602e+06
1,10053_12,128.4460,73059300.0,737183.385744,3.799654e+06
2,10053_18,108.5000,64711200.0,601466.784320,3.006568e+06
3,10138_12,129.0240,71652400.0,699099.199189,3.379573e+06
4,10138_24,142.6480,123897000.0,732120.888877,4.912602e+06


In [20]:
df_peptides = pd.merge(peptides, df_0, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
peptides_PeptideAbundance_updrs.head()

,Peptide,updrs_1_sum
0,AADDTWEPFASGK,6.600976
1,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,6.593625
2,AAFTEC(UniMod_4)C(UniMod_4)QAADK,6.575729
3,AANEVSSADVK,6.573563
4,AATGEC(UniMod_4)TATVGKR,6.511416


In [21]:
df_peptides = pd.merge(peptides, peptides_PeptideAbundance_updrs, on = 'Peptide', how = 'left')
peptides_ft = df_peptides.groupby('visit_id').agg(peptides_updrs_1_min=('updrs_1_sum','min'), peptides_updrs_1_max=('updrs_1_sum','max'),\
                                                          peptides_updrs_1_mean=('updrs_1_sum','mean'), peptides_updrs_1_std=('updrs_1_sum','std'))\
                .reset_index()
peptides_ft

,visit_id,peptides_updrs_1_min,peptides_updrs_1_max,peptides_updrs_1_mean,peptides_updrs_1_std
0,10053_0,6.049037,6.906049,6.527529,0.078700
1,10053_12,5.981534,6.906049,6.525244,0.083481
2,10053_18,5.689507,6.962025,6.514714,0.098449
3,10138_12,5.689507,6.906049,6.500814,0.112576
4,10138_24,5.689507,6.962025,6.504425,0.109622
...,...,...,...,...,...
1108,8699_24,5.760582,6.962025,6.504196,0.107415
1109,942_12,5.760582,6.906049,6.507013,0.100552
1110,942_24,5.760582,6.962025,6.503268,0.104013
1111,942_48,5.760582,6.962025,6.501995,0.104053


In [22]:
c2= pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')

In [23]:
c2.columns=clinical.columns

In [24]:
all_clinical=pd.concat([clinical,c2])

In [25]:
all_clinical2=all_clinical
all_clinical2['upd23b_clinical_state_on_medication']=all_clinical[['upd23b_clinical_state_on_medication']].fillna('Off')
all_clinical2['updrs_4']=all_clinical[['updrs_4']].fillna(0)

In [26]:
all_clinical3=pd.get_dummies(all_clinical2,columns=['upd23b_clinical_state_on_medication'])
all_clinical3=all_clinical3.dropna().reset_index(drop=True)

In [27]:
all_clinical4=all_clinical3.drop(columns=['patient_id','upd23b_clinical_state_on_medication_Off','upd23b_clinical_state_on_medication_On'])
all_clinical4

,visit_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
0,55_0,0,10.0,6.0,15.0,0.0
1,55_3,3,10.0,7.0,25.0,0.0
2,55_6,6,8.0,10.0,34.0,0.0
3,55_9,9,8.0,9.0,30.0,0.0
4,55_12,12,10.0,10.0,41.0,0.0
...,...,...,...,...,...,...
4588,65303_0,0,0.0,2.0,20.0,0.0
4589,65303_36,36,4.0,1.0,26.0,0.0
4590,65405_0,0,5.0,16.0,31.0,0.0
4591,65530_0,0,10.0,6.0,24.0,0.0


In [28]:
peptides[peptides['Peptide']=='GLVSWGNIPC(UniMod_4)GSK']

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
899,55_0,0,55,Q92876,GLVSWGNIPC(UniMod_4)GSK,724168.0
1802,1517_0,0,1517,Q92876,GLVSWGNIPC(UniMod_4)GSK,606324.0
2680,1923_0,0,1923,Q92876,GLVSWGNIPC(UniMod_4)GSK,1161040.0
3587,2660_0,0,2660,Q92876,GLVSWGNIPC(UniMod_4)GSK,808147.0
4467,3636_0,0,3636,Q92876,GLVSWGNIPC(UniMod_4)GSK,330256.0
...,...,...,...,...,...,...
978172,58648_96,96,58648,Q92876,GLVSWGNIPC(UniMod_4)GSK,464939.0
979063,5645_108,108,5645,Q92876,GLVSWGNIPC(UniMod_4)GSK,777961.0
979973,12703_108,108,12703,Q92876,GLVSWGNIPC(UniMod_4)GSK,755370.0
980895,55096_108,108,55096,Q92876,GLVSWGNIPC(UniMod_4)GSK,415757.0


In [29]:
proteins[proteins['UniProt']=='P07333']

,visit_id,visit_month,patient_id,UniProt,NPX
96,55_0,0,55,P07333,54219.0
315,1517_0,0,1517,P07333,31639.9
524,1923_0,0,1923,P07333,43414.7
735,2660_0,0,2660,P07333,74919.3
945,3636_0,0,3636,P07333,45944.4
...,...,...,...,...,...
231557,5645_96,96,5645,P07333,25904.0
231766,58648_96,96,58648,P07333,33764.0
231979,5645_108,108,5645,P07333,20807.3
232193,12703_108,108,12703,P07333,46080.7


In [30]:
# Creating targets
id_cols = ['visit_id','patient_id','visit_month']
target_cols = ['updrs_1','updrs_2','updrs_3','updrs_4']

# Sorting target_data by patient_id and visit_month
clinical.sort_values(['patient_id','visit_month'],inplace=True)
clinical.sort_values(['patient_id','visit_month'],inplace=True)

In [31]:
clinical

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
...,...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,13.0,0.0,Off
2611,65043_54,65043,54,4.0,8.0,11.0,1.0,Off
2612,65043_60,65043,60,6.0,6.0,16.0,1.0,Off
2613,65043_72,65043,72,3.0,9.0,14.0,1.0,Off


In [32]:
clinical1 = clinical.drop(columns=['upd23b_clinical_state_on_medication']).dropna()

In [33]:
# def create_features(peptides_data,protein_data,target_data1):

#     # Processing peptides data and protein data
#     pep_pro_data = peptides_data.merge(protein_data,on = id_cols + ['UniProt'],how = 'left')
#     peptides_list = ['QGVNDNEEGFFSAR','AGLAASLAGPHSIVGR','MELERPGGNEITR','KTSLEDFYLDEER','RYIETDPANRDR','EWVAIESDSVQPVPR','LQDLYSIVR','AIQLTYNPDESSKPNMIDAATLK']
#     parkinson_proteins = set(list(['P37840', 'Q99497', 'Q9NZK5', 'O14773', 'O00443', 'P61077', 'Q13546', 'P53539', 'P26358', 'P02647','P07333']))
#     parkinson_peptides = set(list(['GLVSWGNIPC(UniMod_4)GSK','AGILETGEGEPEA', 'SSDEDAGVDGSAEEVK', 'TGNPNSSAEPGAVVPLGPAGPPAVSGAGPVPTPGEEDLEPDLTSPITATPAVK'])+peptides_list)
    

    
#     pep_pro_data = pep_pro_data[pep_pro_data['UniProt'].isin(parkinson_proteins) | pep_pro_data['Peptide'].isin(parkinson_peptides)].copy()
#     # Some features based on id cols 
#     visit_month_summary = pep_pro_data.groupby('visit_month')[['PeptideAbundance','NPX']].agg(['mean'])
#     visit_month_summary.columns = [i+'_'+j for i,j in visit_month_summary.columns.tolist()]

#     # Calculating ratio columns of NPX and PeptideAbundance
#     for i in ['mean']:
#         visit_month_summary['pepab_to_npx_ratio_'+i] = visit_month_summary[f'PeptideAbundance_{i}']/visit_month_summary[f'NPX_{i}']

#     # Patient level summary
#     patient_summary = pep_pro_data.groupby('patient_id')[['PeptideAbundance','NPX']].agg(['mean'])
#     patient_summary.columns = [i+'_'+j for i,j in patient_summary.columns.tolist()]

#     # Calculating ratio columns of NPX and PeptideAbundance
#     for i in ['mean']:
#         patient_summary['pepab_to_npx_ratio_'+i] = patient_summary[f'PeptideAbundance_{i}']/patient_summary[f'NPX_{i}']

#     # Creating PeptideAbundance / NPX
#     pep_pro_data['pepab_to_npx_ratio'] = pep_pro_data['PeptideAbundance']/pep_pro_data['NPX']
    
#     temp = pep_pro_data.pivot(index= id_cols,columns = 'Peptide',values = ['PeptideAbundance','NPX','pepab_to_npx_ratio'])
#     temp.columns = [j+'_'+i for i,j in temp.columns]
#     temp.reset_index(inplace = True)
#     temp.fillna(0,inplace = True)

#     pep_pro_features = temp.merge(patient_summary.reset_index(),on = 'patient_id').merge(
#         visit_month_summary.reset_index(),on = 'visit_month',suffixes = ('_patient','_visit_month'))
    
#     if target_data1 is None:
#         return pep_pro_features
#     else:
#         pep_pro_features = pep_pro_features.merge(target_data1,on = id_cols,how = 'inner')   
#         return pep_pro_features

In [34]:

# def feature_selector(X,y,k,n_splits,groups):

#     # scaler = StandardScaler()
#     # scaler.fit(X)
#     # X = pd.DataFrame(scaler.transform(X),columns = X.columns.tolist(), index = X.index.tolist())
    
#     # First feature selection using correlation and second using RFECV
#     selector = SelectKBest(score_func=f_regression, k=k)
#     feature_names = X.columns.tolist()

#     # fit selector to the data and transform the feature matrix
#     X_selected = selector.fit_transform(X, y)

#     # get the indices of the selected features
#     selected_indices = selector.get_support(indices=True)
#     selected_features = [feature_names[i] for i in selected_indices]
    
# #     print(selected_features)
# # instantiate a ML model
#     lgbm = lgb.LGBMRegressor(n_jobs=-1)
#     # xgbm = xgb.XGBRegressor()

#     # Initialize GroupKFold with 10 folds
#     kf = GroupKFold(n_splits=n_splits)

#     rfecv = RFECV(estimator=lgbm, min_features_to_select=10,cv = kf.split(X, y, groups), step = 1,scoring = smape_)

#     # fit RFE to the data
#     rfecv.fit(X[selected_features], y)

#     feature_rank = pd.DataFrame({'features' : selected_features,'rank':rfecv.ranking_}).sort_values('rank')
#     selected_features2 = feature_rank[feature_rank['rank'] < 2].features.tolist()
    
#     return selected_features

In [35]:
# # Selecting features
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import lightgbm as lgb
# import tqdm
# import re
# from sklearn.feature_selection import SelectKBest, f_regression, RFE, RFECV

# from itertools import product
# from functools import reduce

# import warnings
# warnings.filterwarnings("ignore", category=UserWarning)
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# warnings.filterwarnings("ignore", category=FutureWarning)

# pd.set_option('display.float_format',lambda x: '%.3f' % x)
# from sklearn.feature_selection import SelectKBest, f_regression, RFE, RFECV
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import make_scorer
# from sklearn.preprocessing import StandardScaler
# import lightgbm as lgbm
# import xgboost as xgbm

# def smape(y_true, y_pred):
#     y_true = 1+y_true
#     y_pred = 1+y_pred
#     return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# smape_ = make_scorer(smape,greater_is_better=False)
# model_data = create_features(peptides,proteins,clinical1)
# model_x = model_data.drop(columns = id_cols +  target_cols).fillna(0).copy()
# model_y = model_data[target_cols].sum(axis = 1).values
# groups = model_data['patient_id']
# sel_features = feature_selector(model_x,model_y,20,10,groups)

In [36]:
clinical[['visit_id']]

,visit_id
0,55_0
1,55_3
2,55_6
3,55_9
4,55_12
...,...
2610,65043_48
2611,65043_54
2612,65043_60
2613,65043_72


In [37]:
# 알고있는 단백질과 펩타이드 리스트
# 가장 많이 나온 단백질, 펩타이드 찾기

#top_proteins = merged['UniProt'].value_counts().head(len(merged['UniProt'].value_counts())).index.tolist()
#top_peptides = merged['Peptide'].value_counts().head(len(merged['Peptide'].value_counts())).index.tolist()
# visit_id를 기준으로 proteins와 peptides를 병합
merged = proteins.merge(peptides, on=['visit_id', 'patient_id', 'visit_month', 'UniProt'], how='inner')
merged=merged.dropna()
# 알고있는 단백질과 펩타이드만 남기기
top_proteins = merged['UniProt'].value_counts().head(13).index.tolist()
top_peptides = merged['Peptide'].value_counts().head(13).index.tolist()
# parkinson_proteins = set(list(['P37840', 'Q99497', 'Q9NZK5', 'O14773', 'O00443', 'P61077', 'Q13546', 'P53539', 'P26358', 'P02647','P07333'])+top_proteins)
# parkinson_peptides = set(list(['GLVSWGNIPC(UniMod_4)GSK','AGILETGEGEPEA', 'SSDEDAGVDGSAEEVK', 'TGNPNSSAEPGAVVPLGPAGPPAVSGAGPVPTPGEEDLEPDLTSPITATPAVK'])+top_peptides)
# Processing peptides data and protein data

peptides_list = ['QGVNDNEEGFFSAR','AGLAASLAGPHSIVGR','MELERPGGNEITR','KTSLEDFYLDEER','RYIETDPANRDR','EWVAIESDSVQPVPR','LQDLYSIVR','AIQLTYNPDESSKPNMIDAATLK']
parkinson_proteins = set(list(['P37840', 'Q99497', 'Q9NZK5', 'O14773', 'O00443', 'P61077', 'Q13546', 'P53539', 'P26358', 'P02647','P07333'])+top_proteins)
parkinson_peptides = set(list(['GLVSWGNIPC(UniMod_4)GSK','AGILETGEGEPEA', 'SSDEDAGVDGSAEEVK', 'TGNPNSSAEPGAVVPLGPAGPPAVSGAGPVPTPGEEDLEPDLTSPITATPAVK'])+peptides_list+top_peptides)
#merged[merged['UniProt'].isin(parkinson_proteins) | merged['Peptide'].isin(parkinson_peptides)].shape[0] == 0:
    # 펩타이드와 프로틴이 없는 경우
    # 가장 많이 나왔던 펩타이드 종류 찾기
    #most_common_peptide = merged['Peptide'].value_counts().index[0]
    # 해당 펩타이드 종류의 값을 가지도록 merged 생성
    #merged = pd.DataFrame({'UniProt': [None], 'embedding': [None]})
#else:
    # 펩타이드와 프로틴이 있는 경우
    #merged = merged[merged['UniProt'].isin(parkinson_proteins) | merged['Peptide'].isin(parkinson_peptides)]


# visit_id를 기준으로 clinical 데이터와 병합
#merged = merged.merge(clinical, on=['visit_id', 'patient_id', 'visit_month'], how='inner')
#dfmerge=merged.drop(columns=['upd23b_clinical_state_on_medication'])
#dfmerge=dfmerge.dropna()
#dfmerge=dfmerge.reset_index(drop=True)

merged = merged[merged['UniProt'].isin(parkinson_proteins) | merged['Peptide'].isin(parkinson_peptides)]
# visit_id를 기준으로 clinical 데이터와 병합
merged = merged.merge(clinical, on=['visit_id', 'patient_id', 'visit_month'], how='inner')
dfmerge=merged.dropna().drop(columns=['upd23b_clinical_state_on_medication'])
dfmerge=dfmerge.reset_index(drop=True)

In [38]:
dfmerge

,visit_id,visit_month,patient_id,UniProt,NPX,Peptide,PeptideAbundance,updrs_1,updrs_2,updrs_3,updrs_4
0,1517_0,0,1517,O14773,17466.3,LFGGNFAHQASVAR,12288.30,11.0,6.0,25.0,5.0
1,1517_0,0,1517,O14773,17466.3,LYQQHGAGLFDVTR,5178.02,11.0,6.0,25.0,5.0
2,1517_0,0,1517,P00734,743551.0,TATSEYQTFFNPR,135958.00,11.0,6.0,25.0,5.0
3,1517_0,0,1517,P01009,13234200.0,FLENEDR,56205.50,11.0,6.0,25.0,5.0
4,1517_0,0,1517,P01009,13234200.0,FLENEDRR,53561.80,11.0,6.0,25.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
134517,55096_108,108,55096,P39060,43632.4,LQDLYSIVR,23576.90,5.0,6.0,46.0,0.0
134518,55096_108,108,55096,P80748,40977.1,FSGSNSGNTATLTISR,40977.10,5.0,6.0,46.0,0.0
134519,55096_108,108,55096,Q12805,966511.0,ADQVC(UniMod_4)INLR,160566.00,5.0,6.0,46.0,0.0
134520,55096_108,108,55096,Q92876,415757.0,GLVSWGNIPC(UniMod_4)GSK,415757.00,5.0,6.0,46.0,0.0


In [39]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import numpy as np
import tensorflow as tf
import random
tf.random.set_seed(ran)
def peptide_embedding_function(df, max_tokens=100, output_mode="int", output_sequence_length=30):
    # Define a vectorizer for the peptide sequences
    vectorizer = TextVectorization(max_tokens=max_tokens, output_mode=output_mode, output_sequence_length=output_sequence_length,random_state=ran)

    # Adapt the vectorizer to the peptide sequences in the DataFrame
    vectorizer.adapt(df['Peptide'].values)

    # Define an embedding layer
    embedding = tf.keras.Sequential([
        vectorizer,
        tf.keras.layers.Embedding(len(vectorizer.get_vocabulary()), 32, mask_zero=True),
        tf.keras.layers.GlobalAveragePooling1D()
    ])

    # Apply the embedding layer to the peptide sequences in the DataFrame
    peptide_embeddings = embedding(df['Peptide'].values)

    return peptide_embeddings.numpy()

def uniprot_embedding_function(df, max_tokens=100, output_mode="int", output_sequence_length=500):
    # Define a vectorizer for the Uniprot sequences
    vectorizer = TextVectorization(max_tokens=max_tokens, output_mode=output_mode, output_sequence_length=output_sequence_length,random_state=ran)

    # Adapt the vectorizer to the Uniprot sequences in the DataFrame
    vectorizer.adapt(df['UniProt'].values)

    # Define an embedding layer
    embedding = tf.keras.Sequential([
        vectorizer,
        tf.keras.layers.Embedding(len(vectorizer.get_vocabulary()), 32, mask_zero=True),
        tf.keras.layers.GlobalAveragePooling1D()
    ])

    # Apply the embedding layer to the Uniprot sequences in the DataFrame
    uniprot_embeddings = embedding(df['UniProt'].values)

    return uniprot_embeddings.numpy()

# apply robust scaling to numerical columns
# 알고있는 단백질과 펩타이드 리스트
dfmerge2 = dfmerge
dfmerge2 = dfmerge2.dropna()
dfmerge2 = dfmerge2.reset_index(drop=True)
num_cols = ['NPX', 'PeptideAbundance', 'visit_month']
dfmerge2[num_cols] = scaler.fit_transform(dfmerge2[num_cols])


# apply peptide_embedding_function and uniprot_embedding_function to peptide and UniProt columns
# peptide_embeddings = peptide_embedding_function(dfmerge)
# uniprot_embeddings = uniprot_embedding_function(dfmerge)



np.random.seed(ran)
# Define a vectorizer for the peptide sequences
vectorizer = TextVectorization(max_tokens=100, output_mode='int', output_sequence_length=500)

# Adapt the vectorizer to the peptide sequences in the DataFrame
vectorizer.adapt(dfmerge['Peptide'].values)

# Define an embedding layer
embedding1 = tf.keras.Sequential([
    vectorizer,
    tf.keras.layers.Embedding(len(vectorizer.get_vocabulary()), 16, mask_zero=True,embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=ran)),
    tf.keras.layers.GlobalAveragePooling1D()
])

# Apply the embedding layer to the peptide sequences in the train DataFrame
peptide_embeddings = embedding1(dfmerge['Peptide'].values).numpy()



vectorizer2 = TextVectorization(max_tokens=100, output_mode='int', output_sequence_length=500)

# Adapt the vectorizer to the peptide sequences in the DataFrame
vectorizer2.adapt(dfmerge['UniProt'].values)

# Define an embedding layer
embedding2 = tf.keras.Sequential([
    vectorizer2,
    tf.keras.layers.Embedding(len(vectorizer.get_vocabulary()), 16, mask_zero=True,embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=ran)),
    tf.keras.layers.GlobalAveragePooling1D()
])

# Apply the embedding layer to the peptide sequences in the train DataFrame
uniprot_embeddings = embedding2(dfmerge['UniProt'].values).numpy()

merged_features = pd.concat([dfmerge[['visit_id']],dfmerge2[num_cols],pd.DataFrame(peptide_embeddings,columns=np.arange(16,32)),pd.DataFrame(uniprot_embeddings)],axis=1)
final_df = pd.concat([merged_features, dfmerge[['updrs_1','updrs_2','updrs_3','updrs_4']]], axis=1)
#test_peptide_embeddings = embedding1(test_df['Peptide'].values).numpy()
#test_peptide_embeddings = embedding1(test_df['Peptide'].values).numpy

In [40]:
#이거바꿨음
final_df=final_df.groupby(by='visit_id').mean()

In [41]:
final_df['visit_id']=final_df.index
final_df = final_df.reset_index(drop=True)
final_df


,NPX,PeptideAbundance,visit_month,16,17,18,19,20,21,22,...,11,12,13,14,15,updrs_1,updrs_2,updrs_3,updrs_4,visit_id
0,2.156253,3.237481,-1.00,0.044031,0.130536,-0.116288,-0.061198,0.139310,-0.024097,0.100755,...,0.033489,0.021468,0.003927,-0.051389,0.007398,3.0,6.0,31.0,0.0,10138_12
1,2.254401,3.293802,-0.50,0.044191,0.131047,-0.116894,-0.061336,0.139849,-0.024156,0.101187,...,0.034999,0.024263,0.000622,-0.051818,0.007304,4.0,7.0,19.0,10.0,10138_24
2,1.912404,2.889630,0.00,0.044138,0.130878,-0.116693,-0.061290,0.139670,-0.024137,0.101044,...,0.033754,0.024515,0.001399,-0.054267,0.007540,5.0,2.0,11.0,0.0,10138_36
3,1.326033,2.036580,1.00,0.044031,0.130536,-0.116288,-0.061198,0.139310,-0.024097,0.100755,...,0.033698,0.021032,0.002223,-0.051357,0.007872,5.0,3.0,29.0,3.0,10541_60
4,1.102426,1.665027,2.00,0.044191,0.131047,-0.116894,-0.061336,0.139849,-0.024156,0.101187,...,0.032425,0.021845,0.002604,-0.050655,0.007494,5.0,9.0,38.0,3.0,10541_84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,2.008116,2.893538,-0.50,0.043646,0.129305,-0.114827,-0.060866,0.138015,-0.023955,0.099714,...,0.037047,0.023789,0.003838,-0.053098,0.007795,5.0,11.0,22.0,0.0,8344_24
437,1.794620,2.693414,0.75,0.043646,0.129305,-0.114827,-0.060866,0.138015,-0.023955,0.099714,...,0.035963,0.021236,0.001365,-0.053665,0.005178,6.0,12.0,21.0,0.0,8344_54
438,2.116518,3.082004,-1.50,0.044031,0.130536,-0.116288,-0.061198,0.139310,-0.024097,0.100755,...,0.034052,0.024710,0.000635,-0.051306,0.006579,17.0,13.0,11.0,4.0,8699_0
439,1.868318,2.808183,-1.00,0.044138,0.130878,-0.116693,-0.061290,0.139670,-0.024137,0.101044,...,0.032361,0.024030,0.001784,-0.051662,0.007184,13.0,15.0,12.0,0.0,8699_12


In [42]:
#from sklearn.decomposition import PCA
#pca = PCA(n_components=16)
#emb_vars = np.arange(64)
#pca_result = pca.fit_transform(final_df[emb_vars])
#print(pca_result.shape)
#pcadf = final_df.drop(columns=np.arange(64))
#pdf=pd.concat([pcadf,pd.DataFrame(pca_result)],axis=1)
#pdf        

In [43]:
#pdf.groupby(by='visit_id').mean()

In [44]:
#날짜통제 clinical[clinical['visit_month']==3]
df_0_1 = clinical[(clinical['visit_month']==3)][['visit_id','updrs_1']].dropna()
df_0_2 = clinical[(clinical['visit_month']==3)][['visit_id','updrs_2']].dropna()
df_0_3 = clinical[(clinical['visit_month']==3)][['visit_id','updrs_3']].dropna()
df_0_4 = all_clinical3[['visit_id','updrs_4']].dropna()

df_proteins = pd.merge(proteins, df_0_1, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs1 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_2, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs2 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_2','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_3, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs3 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_3','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_4, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs4 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_4','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_1, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs1 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_2, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs2 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_2','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_3, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs3 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_3','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_4, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs4 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_4','mean')).reset_index()

df_proteins_fts = [proteins_Uniprot_updrs1, proteins_Uniprot_updrs2, proteins_Uniprot_updrs3, proteins_Uniprot_updrs4]
df_peptides_fts = [peptides_PeptideAbundance_updrs1, peptides_PeptideAbundance_updrs2, peptides_PeptideAbundance_updrs3, peptides_PeptideAbundance_updrs4]
df_lst = [df_0_1, df_0_2, df_0_3, df_0_4]

In [45]:
def features(df, proteins, peptides, classes):
    proteins_npx_ft = proteins.groupby('visit_id').agg(NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std'))\
                    .reset_index()
    peptides_PeptideAbundance_ft = peptides.groupby('visit_id').agg(Abe_min=('PeptideAbundance','min'), Abe_max=('PeptideAbundance','max'),\
                                                                    Abe_mean=('PeptideAbundance','mean'), Abe_std=('PeptideAbundance','std'))\
                    .reset_index()

    df_proteins = pd.merge(proteins, df_proteins_fts[classes], on = 'UniProt', how = 'left')
    proteins_UniProt_ft = df_proteins.groupby('visit_id').agg(proteins_updrs_1_min=('updrs_1_sum','min'), proteins_updrs_1_max=('updrs_1_sum','max'),\
                                                              proteins_updrs_1_mean=('updrs_1_sum','mean'), proteins_updrs_1_std=('updrs_1_sum','std'))\
                    .reset_index()
    df_peptides = pd.merge(peptides, df_peptides_fts[classes], on = 'Peptide', how = 'left')
    peptides_ft = df_peptides.groupby('visit_id').agg(peptides_updrs_1_min=('updrs_1_sum','min'), peptides_updrs_1_max=('updrs_1_sum','max'),\
                                                              peptides_updrs_1_mean=('updrs_1_sum','mean'), peptides_updrs_1_std=('updrs_1_sum','std'))\
                    .reset_index()

    df = pd.merge(df, proteins_npx_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, peptides_PeptideAbundance_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, proteins_UniProt_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, peptides_ft, on = 'visit_id', how = 'left')
    df = df.fillna(df.mean())
    return df

In [46]:
train_all303 = final_df
#pdf = train_all303.groupby(by='visit_id').mean()
#pdf['visit_id'] = pdf.index

In [47]:
#pdf= pdf.reset_index(drop=True)

In [48]:
#pdf

In [49]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
import numpy as np
import warnings
warnings.filterwarnings('ignore')
x=final_df.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4'])
y=final_df.updrs_3
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [LGBMRegressor(random_state=ran), RandomForestRegressor(random_state=ran),XGBRegressor(random_state=100),Lasso(random_state=100)]#Lasso()


params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8,10], 'min_samples_leaf': list(range(1,11,2))},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          #{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]}]
          {'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_3 = {}
oof_preds3 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=123)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model3 = grid_search.best_estimator_
    models_3[i] = best_model3
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model3.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model3.predict(val_X)
        smape_score = smape(val_y, oof_pred[val_idx])
        print(f"Model {i+1}, Fold {fold_idx+1}, SMAPE: {smape_score:.2f}")
    oof_preds3[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

Model 1, Fold 1, SMAPE: 47.59
Model 1, Fold 2, SMAPE: 45.82
Model 1, Fold 3, SMAPE: 44.66
Model 1, Fold 4, SMAPE: 36.23
Model 1, Fold 5, SMAPE: 43.28
Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Best SMAPE: 43.51770145396024
Model 2, Fold 1, SMAPE: 47.02
Model 2, Fold 2, SMAPE: 44.98
Model 2, Fold 3, SMAPE: 45.37
Model 2, Fold 4, SMAPE: 37.26
Model 2, Fold 5, SMAPE: 43.08
Best Parameters: {'max_depth': 3, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
Best SMAPE: 43.538377860404836
Model 3, Fold 1, SMAPE: 48.31
Model 3, Fold 2, SMAPE: 44.33
Model 3, Fold 3, SMAPE: 43.79
Model 3, Fold 4, SMAPE: 38.24
Model 3, Fold 5, SMAPE: 44.62
Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 500}
Best SMAPE: 43.85913555981
Model 4, Fold 1, SMAPE: 45.63
Model 4, Fold 2, SMAPE: 44.97
Model 4, Fold 3, SMAPE: 43.15
Model 4, Fold 4, SMAPE: 34.98
Model 4, Fold 5, SMAPE: 42.75
Best Parameters: {'alpha': 0.01}
Best SMAPE: 42.294382

In [50]:
models_3

{0: LGBMRegressor(learning_rate=0.01, max_depth=3, random_state=99),
 1: RandomForestRegressor(max_depth=3, min_samples_leaf=3, min_samples_split=8,
                       n_estimators=300, random_state=99),
 2: XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=100,
              reg_alpha=0, reg_lambda=1, ...),
 3: Lasso(alpha=0.01, random_state=100)}

In [51]:
#지우지마
x3 = x
y3 = y

In [52]:
combinations=[[0.1, 0.3, 0.3, 0.3], [0.1, 0.2, 0.3, 0.4], [0.1, 0.1, 0.5, 0.3], [0.2, 0.2, 0.2, 0.4], [0.1, 0.1, 0.2, 0.6], [0.2, 0.3, 0.1, 0.4], [0.1, 0.2, 0.1, 0.6], [0.3, 0.2, 0.3, 0.2], [0.1, 0.4, 0.2, 0.3], [0.2, 0.2, 0.4, 0.2], [0.1, 0.1, 0.3, 0.5], [0.2, 0.4, 0.1, 0.3], [0.2, 0.1, 0.3, 0.4], [0.3, 0.3, 0.2, 0.2], [0.1, 0.1, 0.1, 0.7], [0.1, 0.5, 0.2, 0.2], [0.3, 0.1, 0.1, 0.5], [0.3, 0.4, 0.2, 0.1], [0.4, 0.1, 0.1, 0.4], [0.4, 0.2, 0.1, 0.3], [0.2, 0.3, 0.4, 0.1], [0.1, 0.1, 0.4, 0.4], [0.1, 0.2, 0.5, 0.2], [0.2, 0.2, 0.1, 0.5], [0.3, 0.3, 0.3, 0.1], [0.1, 0.1, 0.6, 0.2], [0.2, 0.1, 0.2, 0.5], [0.3, 0.1, 0.3,0.3]]

In [53]:

# 100개의 조합을 생성합니다.
#combinations = []
#for i in range(100):
    
    # 0~1사이의 랜덤한 값을 생성하여 4개의 수로 조합합니다.
  #  comb = [np.round(np.random.uniform(0, 1),2) for i in range(4)]
    # 조합의 합이 1이 되도록 조정합니다.
   # total = sum(comb)
  #  comb = np.round([val/total for val in comb],2)
  #  if sum(comb) ==1:
     #   combinations.append(comb)

# 결과를 출력합니다.
#print(combinations)

In [54]:
# 앙상블 모델 정의
class EnsembleModel(BaseEstimator, RegressorMixin, TransformerMixin):
    
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights
    
    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
    
    def predict(self, X):
        pred = np.zeros(len(X))
        for i, model in enumerate(self.models):
            pred += self.weights[i] * model.predict(X)
        return pred


# 앙상블 모델에 대한 그리드 서치를 수행하며 가중치 튜닝 수행
models_ensemble = [models_3[0], models_3[1], models_3[2], models_3[3]]
params_ensemble = {'weights': combinations}
ensemble_model = EnsembleModel(models_ensemble, weights=[0.25, 0.25, 0.25, 0.25])
grid_search_ensemble = GridSearchCV(ensemble_model, params_ensemble, scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
grid_search_ensemble.fit(X_train, y_train)

best_weights = grid_search_ensemble.best_params_['weights']
best_ensemble_model3 = EnsembleModel(models_ensemble, weights=best_weights)
best_ensemble_model3.fit(X_train, y_train)
y_pred_ensemble = best_ensemble_model3.predict(x_val)
smape_ensemble = smape(y_val, y_pred_ensemble)
print("Best Weights:", best_weights)
print("Best Ensemble SMAPE:",smape_ensemble)

Best Weights: [0.1, 0.1, 0.1, 0.7]
Best Ensemble SMAPE: 40.746052386609875


In [55]:
# up3=pd.DataFrame({'visit_id':tf3['visit_id'],'updrs_3':np.ceil(models_3[0].predict(x3))})
# scaler1=RobustScaler()
# train_1["visit_month"] = train_1["visit_id"].str.split("_", expand=True)[1].astype(int)
# train_all3 = pd.merge(train_1,up3)
# train_all30 = scaler1.fit_transform(train_all3[train_all3.drop(columns=['visit_id','updrs_1']).columns.tolist()])
# train_all300 = pd.DataFrame(train_all30,columns=train_all3.drop(columns=['visit_id','updrs_1']).columns.tolist())
# train_all303=train_all300
# train_all303['visit_id']=train_all3['visit_id']
# tf33=pd.merge(train_all303,df_0_1).reset_index(drop=True)
# tf33

In [56]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
import numpy as np
import warnings
warnings.filterwarnings('ignore')
x=final_df.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4'])
y=final_df.updrs_1
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [LGBMRegressor(random_state=ran), RandomForestRegressor(random_state=ran),XGBRegressor(random_state=100),Lasso(random_state=100)]#Lasso()


params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8,10], 'min_samples_leaf': list(range(1,11,2))},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          #{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]}]
          {'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_1 = {}
oof_preds1 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=100)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model1 = grid_search.best_estimator_
    models_1[i] = best_model1
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model1.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model1.predict(val_X)
        smape_score = smape(val_y, oof_pred[val_idx])
        print(f"Model {i+1}, Fold {fold_idx+1}, SMAPE: {smape_score:.2f}")
    oof_preds1[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

Model 1, Fold 1, SMAPE: 55.55
Model 1, Fold 2, SMAPE: 49.16
Model 1, Fold 3, SMAPE: 56.88
Model 1, Fold 4, SMAPE: 58.60
Model 1, Fold 5, SMAPE: 51.02
Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}
Best SMAPE: 54.24186470226614
Model 2, Fold 1, SMAPE: 54.37
Model 2, Fold 2, SMAPE: 49.33
Model 2, Fold 3, SMAPE: 56.63
Model 2, Fold 4, SMAPE: 58.02
Model 2, Fold 5, SMAPE: 51.75
Best Parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best SMAPE: 54.01844453722539
Model 3, Fold 1, SMAPE: 53.87
Model 3, Fold 2, SMAPE: 49.84
Model 3, Fold 3, SMAPE: 56.69
Model 3, Fold 4, SMAPE: 58.06
Model 3, Fold 5, SMAPE: 53.37
Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
Best SMAPE: 54.36595625598598
Model 4, Fold 1, SMAPE: 55.65
Model 4, Fold 2, SMAPE: 50.66
Model 4, Fold 3, SMAPE: 56.16
Model 4, Fold 4, SMAPE: 58.80
Model 4, Fold 5, SMAPE: 51.24
Best Parameters: {'alpha': 0.1}
Best SMAPE: 54.50239

In [57]:
models_1

{0: LGBMRegressor(learning_rate=0.01, max_depth=5, random_state=99),
 1: RandomForestRegressor(max_depth=5, n_estimators=300, random_state=99),
 2: XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=100,
              reg_alpha=0, reg_lambda=1, ...),
 3: Lasso(alpha=0.1, random_state=100)}

In [58]:
models_ensemble = [models_1[0], models_1[1], models_1[2], models_1[3]]
params_ensemble = {'weights': combinations}
ensemble_model = EnsembleModel(models_ensemble, weights=[0.25, 0.25, 0.25, 0.25])
grid_search_ensemble = GridSearchCV(ensemble_model, params_ensemble, scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
grid_search_ensemble.fit(X_train, y_train)

best_weights = grid_search_ensemble.best_params_['weights']
best_ensemble_model1 = EnsembleModel(models_ensemble, weights=best_weights)
best_ensemble_model1.fit(X_train, y_train)
y_pred_ensemble = best_ensemble_model1.predict(x_val)
smape_ensemble = smape(y_val, y_pred_ensemble)
print("Best Weights:", best_weights)
print("Best Ensemble SMAPE:",smape_ensemble)

Best Weights: [0.1, 0.1, 0.5, 0.3]
Best Ensemble SMAPE: 47.100723197423974


In [59]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
import numpy as np
import warnings
warnings.filterwarnings('ignore')
x=final_df.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4'])
y=final_df.updrs_2
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [LGBMRegressor(random_state=ran), RandomForestRegressor(random_state=ran),XGBRegressor(random_state=100),Lasso(random_state=100)]#Lasso()


params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8,10], 'min_samples_leaf': list(range(1,11,2))},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          #{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]}]
          {'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_2 = {}
oof_preds2 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=100)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model2 = grid_search.best_estimator_
    models_2[i] = best_model2
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model2.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model2.predict(val_X)
        smape_score = smape(val_y, oof_pred[val_idx])
        print(f"Model {i+1}, Fold {fold_idx+1}, SMAPE: {smape_score:.2f}")
    oof_preds2[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)
x4=x

Model 1, Fold 1, SMAPE: 50.06
Model 1, Fold 2, SMAPE: 48.68
Model 1, Fold 3, SMAPE: 48.86
Model 1, Fold 4, SMAPE: 59.78
Model 1, Fold 5, SMAPE: 56.67
Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Best SMAPE: 52.80885157788466
Model 2, Fold 1, SMAPE: 48.55
Model 2, Fold 2, SMAPE: 49.16
Model 2, Fold 3, SMAPE: 51.49
Model 2, Fold 4, SMAPE: 59.37
Model 2, Fold 5, SMAPE: 54.79
Best Parameters: {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 500}
Best SMAPE: 52.67135234220568
Model 3, Fold 1, SMAPE: 49.63
Model 3, Fold 2, SMAPE: 46.72
Model 3, Fold 3, SMAPE: 51.16
Model 3, Fold 4, SMAPE: 58.88
Model 3, Fold 5, SMAPE: 53.81
Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 300}
Best SMAPE: 52.038291497752745
Model 4, Fold 1, SMAPE: 49.16
Model 4, Fold 2, SMAPE: 49.04
Model 4, Fold 3, SMAPE: 48.53
Model 4, Fold 4, SMAPE: 59.02
Model 4, Fold 5, SMAPE: 57.99
Best Parameters: {'alpha': 0.06}
Best SMAPE: 52.745

In [60]:
models_2
models_ensemble = [models_2[0], models_2[1], models_2[2], models_2[3]]
params_ensemble = {'weights': combinations}
ensemble_model = EnsembleModel(models_ensemble, weights=[0.25, 0.25, 0.25, 0.25])
grid_search_ensemble = GridSearchCV(ensemble_model, params_ensemble, scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
grid_search_ensemble.fit(X_train, y_train)

best_weights = grid_search_ensemble.best_params_['weights']
best_ensemble_model2 = EnsembleModel(models_ensemble, weights=best_weights)
best_ensemble_model2.fit(X_train, y_train)
y_pred_ensemble = best_ensemble_model2.predict(x_val)
smape_ensemble = smape(y_val, y_pred_ensemble)
print("Best Weights:", best_weights)
print("Best Ensemble SMAPE:",smape_ensemble)

Best Weights: [0.1, 0.1, 0.6, 0.2]
Best Ensemble SMAPE: 46.67656824487486


# 앙상블 모델에 대한 그리드 서치를 수행하며 가중치 튜닝 수행
models_ensemble = [models_2[0], models_2[1], models_2[2], models_2[3]]
params_ensemble = {'weights': combinations}
ensemble_model = EnsembleModel(models_ensemble, weights=[0.25, 0.25, 0.25, 0.25])
grid_search_ensemble = GridSearchCV(ensemble_model, params_ensemble, scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
grid_search_ensemble.fit(X_train, y_train)

best_weights = grid_search_ensemble.best_params_['weights']
best_ensemble_model2 = EnsembleModel(models_ensemble, weights=best_weights)
best_ensemble_model2.fit(X_train, y_train)
y_pred_ensemble = best_ensemble_model2.predict(x_val)
smape_ensemble = smape(y_val, y_pred_ensemble)
print("Best Weights:", best_weights)
print("Best Ensemble SMAPE:",smape_ensemble)

In [61]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
import numpy as np
import warnings
warnings.filterwarnings('ignore')
x=final_df.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4'])
y=final_df.updrs_4
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
#모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [LGBMRegressor(random_state=ran), RandomForestRegressor(random_state=ran),XGBRegressor(random_state=100),Lasso(random_state=100)]#Lasso()


params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8,10], 'min_samples_leaf': list(range(1,11,2))},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          #{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]}]
          {'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}]
 #SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape, greater_is_better=False)
 #각 모델에 대한 그리드 서치와 OOF 예측 수행
models_4 = {}
oof_preds4 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=123)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model4 = grid_search.best_estimator_
    models_4[i] = best_model4
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model4.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model4.predict(val_X)
        smape_score = smape(val_y, oof_pred[val_idx])
        print(f"Model {i+1}, Fold {fold_idx+1}, SMAPE: {smape_score:.2f}")
    oof_preds4[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

Model 1, Fold 1, SMAPE: 136.82
Model 1, Fold 2, SMAPE: 133.36
Model 1, Fold 3, SMAPE: 128.23
Model 1, Fold 4, SMAPE: 126.32
Model 1, Fold 5, SMAPE: 140.68
Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}
Best SMAPE: 133.08217499912055
Model 2, Fold 1, SMAPE: 134.82
Model 2, Fold 2, SMAPE: 131.03
Model 2, Fold 3, SMAPE: 130.01
Model 2, Fold 4, SMAPE: 125.51
Model 2, Fold 5, SMAPE: 140.75
Best Parameters: {'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 300}
Best SMAPE: 132.42434341228062
Model 3, Fold 1, SMAPE: 141.63
Model 3, Fold 2, SMAPE: 133.46
Model 3, Fold 3, SMAPE: 131.37
Model 3, Fold 4, SMAPE: 128.95
Model 3, Fold 5, SMAPE: 142.14
Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
Best SMAPE: 135.50767992139748
Model 4, Fold 1, SMAPE: 133.56
Model 4, Fold 2, SMAPE: 136.39
Model 4, Fold 3, SMAPE: 126.65
Model 4, Fold 4, SMAPE: 128.05
Model 4, Fold 5, SMAPE: 140.32
Best Parameters: {'alpha': 0.

In [62]:
models_4

{0: LGBMRegressor(learning_rate=0.01, max_depth=5, random_state=99),
 1: RandomForestRegressor(max_depth=7, min_samples_split=8, n_estimators=300,
                       random_state=99),
 2: XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=100,
              reg_alpha=0, reg_lambda=1, ...),
 3: Lasso(alpha=0.01, random_state=100)}

In [63]:

models_ensemble = [models_4[0], models_4[1], models_4[2], models_4[3]]
params_ensemble = {'weights': combinations}
ensemble_model = EnsembleModel(models_ensemble, weights=[0.25, 0.25, 0.25, 0.25])
grid_search_ensemble = GridSearchCV(ensemble_model, params_ensemble, scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
grid_search_ensemble.fit(X_train, y_train)

best_weights = grid_search_ensemble.best_params_['weights']
best_ensemble_model4 = EnsembleModel(models_ensemble, weights=best_weights)
best_ensemble_model4.fit(X_train, y_train)
y_pred_ensemble = best_ensemble_model4.predict(x_val)
smape_ensemble = smape(y_val, y_pred_ensemble)
print("Best Weights:", best_weights)
print("Best Ensemble SMAPE:",smape_ensemble)

Best Weights: [0.2, 0.3, 0.1, 0.4]
Best Ensemble SMAPE: 137.0366569311171


In [64]:
af=pd.DataFrame({'updrs_4':np.ceil(best_ensemble_model4.predict(x))})
af.value_counts()

updrs_4
3.0        185
2.0        156
4.0         75
5.0         19
6.0          4
7.0          1
8.0          1
dtype: int64

In [65]:
y.value_counts()

0.0     225
3.0      34
5.0      31
4.0      31
2.0      29
1.0      22
6.0      17
7.0      15
8.0      11
9.0       8
10.0      5
13.0      4
12.0      3
14.0      2
11.0      2
20.0      1
16.0      1
Name: updrs_4, dtype: int64

In [66]:
af

,updrs_4
0,2.0
1,2.0
2,2.0
3,3.0
4,4.0
...,...
436,3.0
437,3.0
438,3.0
439,2.0


In [67]:
for i in range(len(y)):
    if af.values[i][0]>6 and y[i]>=7:
        print(af.values[i][0],y[i])

7.0 14.0
8.0 20.0


In [68]:
for i in range(len(y)):
    if af.values[i][0]>6 and y[i]<=6:
        print(af.values[i][0],y[i])

In [69]:
model = {}
target = ["updrs_3", "updrs_1", "updrs_2", "updrs_4"]
mo=[best_model1,best_model2,best_model3]
model["updrs_1"]=models_1[0]
model["updrs_2"]=models_2[1]
model["updrs_3"]=best_ensemble_model3
model["updrs_4"]=best_ensemble_model4

In [70]:
test

,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0
5,50423_0,0,50423,updrs_2,50423_0_updrs_2,0
6,50423_0,0,50423,updrs_3,50423_0_updrs_3,0
7,50423_0,0,50423,updrs_4,50423_0_updrs_4,0
8,3342_6,6,3342,updrs_1,3342_6_updrs_1,6
9,3342_6,6,3342,updrs_2,3342_6_updrs_2,6


In [71]:
#change
def mergeall(my_train, test_proteins, test_peptides):
    merged3 = test_proteins.merge(test_peptides, on=['visit_id', 'patient_id', 'visit_month', 'UniProt'], how='inner')
    merged20 = test_proteins.merge(test_peptides,on=['visit_id', 'patient_id', 'visit_month', 'UniProt'],how='outer')
    merged20 = merged20.merge(my_train, on=['visit_id', 'patient_id', 'visit_month'], how='outer')
    #merged3[['UniProt','Peptide']]=merged3[['UniProt','Peptide']].fillna
    merged3[['NPX','PeptideAbundance']] = merged3[['NPX','PeptideAbundance']].fillna(merged3.mean())
    merged3=merged3.dropna()
   # merged20=merged3
# 알고있는 단백질과 펩타이드만 남기기
    merged3 = merged3[merged3['UniProt'].isin(parkinson_proteins) | merged3['Peptide'].isin(parkinson_peptides)][['visit_id','visit_month','patient_id','UniProt','NPX','Peptide','PeptideAbundance']]
    #merged30 = merged20[~merged20['visit_id'].isin(test['visit_id'].tolist())&(~(merged3['UniProt'].isin(parkinson_proteins)) | ~(merged3['Peptide'].isin(parkinson_peptides)))][['visit_id','visit_month','patient_id','UniProt','NPX','Peptide','PeptideAbundance','group_key_x']]
# merged20 데이터프레임에서 test 데이터프레임의 visit_id를 포함하는 row를 제외하고 필터링
   # merged30 = merged20[~merged20['visit_id'].isin(merged3['visit_id'].tolist())][['visit_id','visit_month','patient_id','UniProt','NPX','Peptide','PeptideAbundance','group_key_x']]
    # visit_id를 기준으로 clinical 데이터와 병합
    dfmerge3 = merged3.merge(my_train, on=['visit_id', 'patient_id', 'visit_month'], how='inner')
    dfmerge3 = dfmerge3.fillna(0)
    dfmerge3 = dfmerge3.reset_index(drop=True)
    num_cols = ['NPX', 'PeptideAbundance', 'visit_month']
    dfmerge3[num_cols] = scaler.transform(dfmerge3[num_cols])
    
    # apply peptide_embedding_function and uniprot_embedding_function to peptide and UniProt columns
    peptide_embeddings = embedding1(dfmerge3['Peptide'].values).numpy()
    uniprot_embeddings = embedding2(dfmerge3['UniProt'].values).numpy()
    merged_features = pd.concat([dfmerge3[['visit_id']],dfmerge3[num_cols],pd.DataFrame(peptide_embeddings,columns=np.arange(16,32)),pd.DataFrame(uniprot_embeddings)],axis=1)
    final_df = merged_features
    
    #emb_vars = np.arange(64)
    #pca_result = pca.transform(final_df[emb_vars])
    #pcadf = final_df.drop(columns=np.arange(64))
    #pdf = pd.concat([pcadf, pd.DataFrame(pca_result)], axis=1)
    merged30 = merged20[~merged20['visit_id'].isin(final_df['visit_id'].tolist())][['visit_id','visit_month','patient_id','UniProt','NPX','Peptide','PeptideAbundance']]
    dfmerge30 = merged30.merge(my_train, on=['visit_id', 'patient_id', 'visit_month'], how='right')
    dfmerge30[np.arange(32)] = final_df[np.arange(32)].mean()
    dfmerge30 = dfmerge30.reset_index(drop=True)
    if len(dfmerge30) != 0:
        dfmerge30[num_cols] = scaler.transform(dfmerge30[num_cols])
    
    df = pd.concat([final_df, dfmerge30]).sort_values('visit_id').reset_index(drop=True)
    df = df.groupby(by='visit_id').mean()
    df['visit_id'] = df.index
    df = df.reset_index(drop=True)
    df.fillna(0)
    df = df.fillna(df.mean())
    return df

In [72]:
# dfff=[]
# test = test.fillna(0)
# train_all303 = mergeall(test,proteins_test,peptides_test)
# ffinal = train_all303
#     #ffinal = ffinal.rename(columns={'group_key_x': 'group_key'})
#     #ffinal['visit_id'] = ffinal.index
    
# dropcol=['visit_id','patient_id','UniProt','Peptide','row_id','updrs_test','group_key']
#    # df = ffinal[['visit_id']].drop_duplicates()
# df = ffinal[['visit_id']]
# df
# for u in target:   
#         # Here is where we will save the final results
#     df['result_' + str(u)] = 0
#         # Predict  
#     if u == 'updrs_1':
#         df['result_' + str(u)] = (model['updrs_1'].predict(ffinal.drop(columns=dropcol).fillna(0))).tolist()
#     if u == 'updrs_2':
#         df['result_' + str(u)] = (model['updrs_2'].predict(ffinal.drop(columns=dropcol).fillna(0))).tolist()
#     if u == 'updrs_3':
#         df['result_' + str(u)] = (model['updrs_3'].predict(ffinal.drop(columns=dropcol).fillna(0))).tolist()
#     if u == 'updrs_4':
            #for i in range(len(np.ceil(model['updrs_4'].predict(train_all303.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4']))))):
                #if np.ceil(model['updrs_4'].predict(train_all303.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4'])))[i] >7:
                    #dfff.append(np.ceil(model['updrs_4'].predict(train_all303.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4'])))[i]+5)
                #else:
                    #dfff.append(0)

#             #my_train['result_' + str(u)] = dfff
#         df['result_' + str(u)] = 0
#         #df['group_key'] = df['visit_id'].str.extract('(\d+)$', expand=False)
# df = df.groupby(by='visit_id').mean().reset_index()
# df['result_updrs_1']=np.ceil(df['result_updrs_1'])
# df['result_updrs_2']=np.ceil(df['result_updrs_2'])
# df['result_updrs_3']=np.ceil(df['result_updrs_3'])
# df            

In [73]:
# dfff=[]
# test = test.fillna(0)
# train_all303 = mergeall(test,proteins_test,peptides_test)
# ffinal = train_all303
#     #ffinal = ffinal.rename(columns={'group_key_x': 'group_key'})
#     #ffinal['visit_id'] = ffinal.index
    
# dropcol=['visit_id','patient_id','UniProt','Peptide','row_id','updrs_test','group_key']
#    # df = ffinal[['visit_id']].drop_duplicates()
# df = ffinal[['visit_id']]
# for u in target:   
#         # Here is where we will save the final results
#     df['result_' + str(u)] = 0
#         # Predict  
#     if u == 'updrs_1':
#         df['result_' + str(u)] = (model['updrs_1'].predict(ffinal.drop(columns=dropcol).fillna(0))).tolist()
#     if u == 'updrs_2':
#         df['result_' + str(u)] = (model['updrs_2'].predict(ffinal.drop(columns=dropcol).fillna(0))).tolist()
#     if u == 'updrs_3':
#         df['result_' + str(u)] = (model['updrs_3'].predict(ffinal.drop(columns=dropcol).fillna(0))).tolist()
#     if u == 'updrs_4':
#             #for i in range(len(np.ceil(model['updrs_4'].predict(train_all303.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4']))))):
#                 #if np.ceil(model['updrs_4'].predict(train_all303.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4'])))[i] >7:
#                     #dfff.append(np.ceil(model['updrs_4'].predict(train_all303.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4'])))[i]+5)
#                 #else:
#                     #dfff.append(0)

#             #my_train['result_' + str(u)] = dfff
#         df['result_' + str(u)] = 0
#         #df['group_key'] = df['visit_id'].str.extract('(\d+)$', expand=False)
# df = df.groupby(by='visit_id').mean().reset_index()
# df['result_updrs_1']=np.ceil(df['result_updrs_1'])
# df['result_updrs_2']=np.ceil(df['result_updrs_2'])
# df['result_updrs_3']=np.ceil(df['result_updrs_3'])
# df.drop_duplicates()            
# result = pd.DataFrame()

# for m in [0, 6, 12, 24]:
#     for u in [1, 2, 3, 4]:

#         temp = df[["visit_id", "result_updrs_" + str(u)]]
#         temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
#         temp["rating"] = temp["result_updrs_" + str(u)]
#         temp = temp [['prediction_id', 'rating']]

#         result = result.append(temp)            
# result = result.drop_duplicates(subset=['prediction_id', 'rating'])
# df

In [74]:

def get_predictions(my_train, test_proteins,test_peptides, model):
    # Forecast
    dfff=[]
    my_train = my_train.fillna(0)
    train_all303 = mergeall(my_train,test_proteins,test_peptides)
    ffinal = train_all303
    #ffinal = ffinal.rename(columns={'group_key_x': 'group_key'})
    #ffinal['visit_id'] = ffinal.index
    
   # dropcol=['visit_id','patient_id','UniProt','Peptide','row_id','updrs_test']
    dropcol=['visit_id','patient_id']
   # df = ffinal[['visit_id']].drop_duplicates()
    df = ffinal[['visit_id']]
    for u in target:   
        # Here is where we will save the final results
        df['result_' + str(u)] = 0
        # Predict  
        if u == 'updrs_1':
            df['result_' + str(u)] = (model['updrs_1'].predict(ffinal.drop(columns=dropcol).fillna(0))).tolist()
        if u == 'updrs_2':
            df['result_' + str(u)] = (model['updrs_2'].predict(ffinal.drop(columns=dropcol).fillna(0))).tolist()
        if u == 'updrs_3':
            df['result_' + str(u)] = (model['updrs_3'].predict(ffinal.drop(columns=dropcol).fillna(0))).tolist()
        if u == 'updrs_4':
            for i in range(len(np.ceil(model['updrs_4'].predict(train_all303.drop(columns=dropcol).fillna(0))))):
                if np.ceil(model['updrs_4'].predict(train_all303.drop(columns=dropcol).fillna(0)))[i] >100:
                    dfff.append(np.ceil(model['updrs_4'].predict(train_all303.drop(columns=dropcol).fillna(0)))[i])
                else:
                    dfff.append(0)
            df['result_' + str(u)] = dfff
            #df['result_' + str(u)] = 0
        #df['group_key'] = df['visit_id'].str.extract('(\d+)$', expand=False)
    df = df.groupby(by='visit_id').mean().reset_index()
    
    df['result_updrs_1']=np.trunc(df['result_updrs_1'])-3
    df['result_updrs_2']=np.round(df['result_updrs_2'],0)-3
    df['result_updrs_3']=np.ceil(df['result_updrs_3'])
            
 # Format for final submission
    result = pd.DataFrame()
    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3, 4]:
            temp = df[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp [['prediction_id', 'rating']]
            result = result.append(temp)            
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])
    return result

In [75]:
import amp_pd_peptide
env = amp_pd_peptide.make_env()
iter_test = env.iter_test()

In [76]:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    display(test)
    
    
    display(mergeall(test,test_proteins,test_peptides))    
    result = get_predictions(test,test_proteins,test_peptides, model)

    #result=result.sort_values(by='prediction_id').reset_index(drop=True)
    env.predict(result)   # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,visit_id,visit_month,patient_id,updrs_test,row_id
0,3342_0,0,3342,updrs_1,3342_0_updrs_1
1,3342_0,0,3342,updrs_2,3342_0_updrs_2
2,3342_0,0,3342,updrs_3,3342_0_updrs_3
3,3342_0,0,3342,updrs_4,3342_0_updrs_4
4,50423_0,0,50423,updrs_1,50423_0_updrs_1
5,50423_0,0,50423,updrs_2,50423_0_updrs_2
6,50423_0,0,50423,updrs_3,50423_0_updrs_3
7,50423_0,0,50423,updrs_4,50423_0_updrs_4


,NPX,PeptideAbundance,visit_month,16,17,18,19,20,21,22,...,8,9,10,11,12,13,14,15,patient_id,visit_id
0,2.581537,3.272004,-1.5,0.046273,0.136933,-0.123372,-0.062836,0.145208,-0.02417,0.105761,...,0.014185,-0.030758,-0.067316,0.035672,0.025339,-0.001463,-0.053917,0.012832,3342.0,3342_0
1,2.581537,3.272004,-1.5,0.046273,0.136933,-0.123372,-0.062836,0.145208,-0.02417,0.105761,...,0.014185,-0.030758,-0.067316,0.035672,0.025339,-0.001463,-0.053917,0.012832,50423.0,50423_0


,visit_id,visit_month,patient_id,updrs_test,row_id
0,3342_6,6,3342,updrs_1,3342_6_updrs_1
1,3342_6,6,3342,updrs_2,3342_6_updrs_2
2,3342_6,6,3342,updrs_3,3342_6_updrs_3
3,3342_6,6,3342,updrs_4,3342_6_updrs_4
4,50423_6,6,50423,updrs_1,50423_6_updrs_1
5,50423_6,6,50423,updrs_2,50423_6_updrs_2
6,50423_6,6,50423,updrs_3,50423_6_updrs_3
7,50423_6,6,50423,updrs_4,50423_6_updrs_4


,NPX,PeptideAbundance,visit_month,16,17,18,19,20,21,22,...,8,9,10,11,12,13,14,15,patient_id,visit_id
0,1.762337,2.262131,-1.25,0.046724,0.138398,-0.125126,-0.063233,0.146776,-0.024359,0.107012,...,0.012965,-0.030235,-0.066899,0.037257,0.026167,0.000607,-0.05548,0.011904,3342.0,3342_6
1,1.762337,2.262131,-1.25,0.046724,0.138398,-0.125126,-0.063233,0.146776,-0.024359,0.107012,...,0.012965,-0.030235,-0.066899,0.037257,0.026167,0.000607,-0.05548,0.011904,50423.0,50423_6


In [77]:
result.sort_values(by='prediction_id').reset_index(drop=True)

,prediction_id,rating
0,3342_6_updrs_1_plus_0_months,6.0
1,3342_6_updrs_1_plus_12_months,6.0
2,3342_6_updrs_1_plus_24_months,6.0
3,3342_6_updrs_1_plus_6_months,6.0
4,3342_6_updrs_2_plus_0_months,6.0
5,3342_6_updrs_2_plus_12_months,6.0
6,3342_6_updrs_2_plus_24_months,6.0
7,3342_6_updrs_2_plus_6_months,6.0
8,3342_6_updrs_3_plus_0_months,23.0
9,3342_6_updrs_3_plus_12_months,23.0


In [78]:
get_predictions(test,test_proteins,test_peptides, model)

,prediction_id,rating
0,3342_6_updrs_1_plus_0_months,6.0
1,50423_6_updrs_1_plus_0_months,6.0
0,3342_6_updrs_2_plus_0_months,6.0
1,50423_6_updrs_2_plus_0_months,6.0
0,3342_6_updrs_3_plus_0_months,23.0
1,50423_6_updrs_3_plus_0_months,23.0
0,3342_6_updrs_4_plus_0_months,0.0
1,50423_6_updrs_4_plus_0_months,0.0
0,3342_6_updrs_1_plus_6_months,6.0
1,50423_6_updrs_1_plus_6_months,6.0


In [79]:
test

,visit_id,visit_month,patient_id,updrs_test,row_id
0,3342_6,6,3342,updrs_1,3342_6_updrs_1
1,3342_6,6,3342,updrs_2,3342_6_updrs_2
2,3342_6,6,3342,updrs_3,3342_6_updrs_3
3,3342_6,6,3342,updrs_4,3342_6_updrs_4
4,50423_6,6,50423,updrs_1,50423_6_updrs_1
5,50423_6,6,50423,updrs_2,50423_6_updrs_2
6,50423_6,6,50423,updrs_3,50423_6_updrs_3
7,50423_6,6,50423,updrs_4,50423_6_updrs_4
